In [75]:
import numpy as np
import pandas as pd
import tqdm
import re
from lightfm import LightFM

In [99]:
from io import BytesIO
from PIL import Image
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds
from sklearn import preprocessing
from sklearn.cluster import KMeans

# Домашнее задание

Data contains the book rating information. Ratings (Book-Rating) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.

### 1. Реализовать персональный топ  - принимает на вход возраст и локацию, на выходе персональный топ   - 1 балл

Персональный топ - это топ товаров по похожим возрасту/интересам/локации. Как сделать? Разбить на сегменты по выбраным признакам. Топ делать по книгам с хорошим средним рейтингом.

### 2. На основе метода кластеризации похожих пользователей построить рекомендации (Слайд 27) - 3 балла

Нужно топ-10 рекомендаций с самой высокой оценкой. Считаем среднюю оценку для каждой книги по кластеру и выводим топ-10 книг.

### 3. Совстречаемость - 3 балла

В совстречаемости также учитывать оценки. Вес пары книг встретившихся у пользователя - полусумма их оценок.

### 4. Коллаборативная фильтрация - 3 балла

Коллаборативную фильтрацию реализовывать как на слайде 51 презентации, посоветовав каждому пользователю топ-10 книг с самой высокой оценкой. Сделать рекомендации User-based и Item-based и сравнить.

Если совсем сложно - можно сделать как в семинарской части, поставив оценку "0", если рейтинг < 5 и "1" - в противном случае. Тогда максимум за это - 1 балл. Реализовать U2I и I2I рекомендации.

### Примечание:

Так как пользователей много - можно зафиксировать несколько произвольных и для них уже составлять рекомендации

Работоспособность I2I можно проверять на известных книгах (Гарри Поттер, Властелин Колец, Интервью с вампиром, Код-Да-Винчи, Маленький Принц)

Рейтинг книг обязательно нужно учитывать

Не забываем также предобработать данные - выкинуть выбросы-пользователей и выбросы-книги.

Выводить в качестве рекомендаций лучше названия книг, картинки (если они есть) и соответствующие метрики близости.

In [77]:
books = pd.read_csv("BX-Books.csv")
books.head(2)

C:\Users\baben_bakg1j1\AppData\Local\Temp\ipykernel_1332\3153319633.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("BX-Books.csv")


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [78]:
interactions = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding = "ISO-8859-1")
interactions = interactions[interactions["Book-Rating"] != 0]
interactions.head(2)

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3


In [79]:
books_meets = interactions.groupby("ISBN")["User-ID"].count().reset_index().rename(columns={"User-ID": "user_num"})
books_meets.head(2)

,ISBN,user_num
0,0330299891,1
1,0375404120,1


In [80]:
user_meets = interactions.groupby("User-ID")["ISBN"].count().reset_index().rename(columns={"ISBN": "books_num"})
user_meets.head(2)

,User-ID,books_num
0,8,7
1,9,1


In [81]:
interactions = interactions.merge(books_meets, on=["ISBN"]).merge(user_meets, on=["User-ID"])
interactions.head(2)

,User-ID,ISBN,Book-Rating,user_num,books_num
0,276726,0155061224,5,1,1
1,276729,052165615X,3,1,2


In [82]:
interactions_1 = interactions.copy()

In [83]:
interactions = interactions[(interactions["user_num"] > 5) & 
                            (interactions["books_num"] > 5) &
                            (interactions["books_num"] < 200)]
interactions.head(2)

,User-ID,ISBN,Book-Rating,user_num,books_num
5,86583,3404139178,9,8,18
6,86583,3453061187,8,13,18


In [84]:
users = pd.read_csv('BX-Users.csv', delimiter=';', encoding = 'ISO-8859-1')
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [85]:
interactions = interactions.merge(books[["ISBN", "Image-URL-M", "Book-Title"]].rename(
    columns={"Image-URL-M": "picture_url"}), on=["ISBN"])
interactions.head(2)

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman
1,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman


In [86]:
le = preprocessing.LabelEncoder()

In [87]:
interactions["product_id"] = le.fit_transform(interactions["ISBN"])
interactions["vid"] = le.fit_transform(interactions["User-ID"])
interactions.head(2)

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,product_id,vid
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,3445
1,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,5248


In [88]:
csr_rates = coo_matrix((interactions["Book-Rating"], (interactions["vid"], interactions["product_id"])), 
                            shape=(len(set(interactions["vid"])), len(set(interactions["product_id"]))))

### Ищем id нужных книг

In [89]:
for i, j in interactions[["product_id", "Book-Title"]].drop_duplicates().values:
    if "David Copperfield" in j:
        print("idx:", i, "\tBook Title:", j)

idx: 10174 	Book Title: David Copperfield (Wordsworth Classics)
idx: 1074 	Book Title: David Copperfield (Penguin Classics)


In [90]:
df = interactions_1.drop(['user_num', 'books_num'], axis=1).iloc[:50000]

### 1. Реализовать персональный топ  - принимает на вход возраст и локацию, на выходе персональный топ   - 1 балл

Персональный топ - это топ товаров по похожим возрасту/интересам/локации. Как сделать? Разбить на сегменты по выбраным признакам. Топ делать по книгам с хорошим средним рейтингом.

In [91]:
#Пример введенных значений
age = 18
location = "stockton, california, usa"

In [92]:
#Беру += 3 года, так как очень мало людей ровно того же возраста и из того же места
users_top = users[(users["Age"] >= age - 3) & (users["Age"] <= age + 3)]
users_top = users_top[users_top["Location"] == location]
users_top.head()

,User-ID,Location,Age
1,2,"stockton, california, usa",18.0
32256,32257,"stockton, california, usa",16.0
79350,79351,"stockton, california, usa",19.0
106117,106118,"stockton, california, usa",19.0
156923,156924,"stockton, california, usa",15.0


In [100]:
user_ids = users_top["User-ID"].tolist()
print(user_ids)

[2, 32257, 79351, 106118, 156924, 199767, 211403, 232865]


In [101]:
similar_books = interactions_1[interactions_1["User-ID"].isin(user_ids)]
similar_books = similar_books[similar_books['Book-Rating'] > 6]
similar_books.head()

,User-ID,ISBN,Book-Rating,user_num,books_num
378741,79351,1854582747,10,3,1
396033,156924,0395645662,9,7,1
401349,232865,0452274427,8,13,1


In [102]:
print("Топ рекомендованных книг: " + str(similar_books['ISBN'].value_counts().head(5).index.tolist()) + " (ISBN)")

Топ рекомендованных книг: ['1854582747', '0395645662', '0452274427'] (ISBN)


### 2. На основе метода кластеризации похожих пользователей построить рекомендации (Слайд 27) - 3 балла

Нужно топ-10 рекомендаций с самой высокой оценкой. Считаем среднюю оценку для каждой книги по кластеру и выводим топ-10 книг.

In [103]:
df_to_cluster = interactions_1.drop(['user_num', 'books_num'], axis=1)
df_to_cluster.head(2)

,User-ID,ISBN,Book-Rating
0,276726,0155061224,5
1,276729,052165615X,3


In [104]:
kmeans = KMeans(n_clusters=10)
kmeans.fit(df_to_cluster[['User-ID', 'Book-Rating']])

KMeans(n_clusters=10)

In [105]:
df_to_cluster['cluster'] = kmeans.labels_
df_to_cluster.head(3)

,User-ID,ISBN,Book-Rating,cluster
0,276726,0155061224,5,5
1,276729,052165615X,3,5
2,276729,0521795028,6,5


In [106]:
mean_ratings = df_to_cluster.groupby(['cluster', 'ISBN'])['Book-Rating'].mean().reset_index()
top_10 = mean_ratings.sort_values(by='Book-Rating', ascending=False).groupby('cluster').head(10)

In [107]:
#for i in range(10):
#    print(f"cluster {i}: ")
#    print(top_10[top_10["cluster"]==i]['ISBN'].tolist())

In [108]:
user_id = 276729
cluster = df_to_cluster[df_to_cluster['User-ID']==user_id].iloc[0]['cluster']
print("Реккомендации для пользователя: " + str(top_10[top_10["cluster"]==cluster]['ISBN'].tolist()))

Реккомендации для пользователя: ['3250100692', '3257060580', '3257062303', '3257205155', '3257218370', '3257224176', '3257230885', '3100922484', '3100970616', '3123512304']


### 3. Совстречаемость - 3 балла

В совстречаемости также учитывать оценки. Вес пары книг встретившихся у пользователя - полусумма их оценок.

In [109]:
ocur = interactions_1.drop(['user_num', 'books_num'], axis=1)
ocur.head()

,User-ID,ISBN,Book-Rating
0,276726,0155061224,5
1,276729,052165615X,3
2,276729,0521795028,6
3,276736,3257224281,8
4,86583,3257224281,6


In [110]:
class Recommendations:
    def __init__(self, data):
        self.data = data
        self.user_item_ratings = self._prepare_data()
        self.cooc_rec = None
    
    def _prepare_data(self):
        user_item_ratings = self.data.rename(columns={"User-ID": "user_id", "ISBN": "item_id", "Book-Rating": "rating"})
        return user_item_ratings
    
    def cooccurrence_count(self):
        # Фильтр для пользоватлей, купивших только одну книгу
        user_item_counts = self.user_item_ratings.groupby('user_id').size()
        valid_users = user_item_counts[user_item_counts > 1].index
        valid_ratings = self.user_item_ratings[self.user_item_ratings['user_id'].isin(valid_users)]
        
        cooc = {}
        for user, group in tqdm.tqdm_notebook(valid_ratings.groupby('user_id')):
            items = group[['item_id', 'rating']].values
            for i in range(len(items)):
                for j in range(i + 1, len(items)):
                    pair = tuple(sorted([items[i][0], items[j][0]]))
                    weight = (items[i][1] + items[j][1]) / 2.0
                    cooc[pair] = cooc.get(pair, 0) + weight
        
        cooc_list = [(pair[0], pair[1], count) for pair, count in cooc.items() if count > 1]
        self.cooc_rec = pd.DataFrame(cooc_list, columns=["item1", "item2", "measure"])
    
    def get_recommendations(self, item_id, show=False):
        if self.cooc_rec is None:
            print("Запустите cooccurrence_count() сначала.")
            return
        
        recs = self.cooc_rec[self.cooc_rec["item1"] == item_id]\
                            .sort_values("measure", ascending=False)\
                            .head(10)
        
        print("Для книги:")
        print(item_id)
        print("Такие рекомендации:")
        print(recs["item2"].values)


In [35]:
recommender = Recommendations(ocur)
recommender.cooccurrence_count()

C:\Users\baben_bakg1j1\AppData\Local\Temp\ipykernel_20464\3603846240.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user, group in tqdm.tqdm_notebook(valid_ratings.groupby('user_id')):


  0%|          | 0/32423 [00:00<?, ?it/s]

In [41]:
recommender.get_recommendations('052165615X')

For item:
052165615X
Recommendations:
['0521795028']


### 4. Коллаборативная фильтрация - 3 балла

Коллаборативную фильтрацию реализовывать как на слайде 51 презентации, посоветовав каждому пользователю топ-10 книг с самой высокой оценкой. Сделать рекомендации User-based и Item-based и сравнить.

Если совсем сложно - можно сделать как в семинарской части, поставив оценку "0", если рейтинг < 5 и "1" - в противном случае. Тогда максимум за это - 1 балл. Реализовать U2I и I2I рекомендации.

In [112]:
df.head(2)

,User-ID,ISBN,Book-Rating
0,276726,0155061224,5
1,276729,052165615X,3


In [113]:
user_book_matrix = df.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating')
user_book_matrix = user_book_matrix.fillna(0)
user_similarity = cosine_similarity(user_book_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_book_matrix.index, columns=user_book_matrix.index)

def get_top_n_recommendations_user_based(user_id, top_n=10):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]
    
    user_books = user_book_matrix.loc[user_id]
    unrated_books = user_books[user_books == 0]
    
    recommendations = []
    for similar_user_id, similarity_score in similar_users.iteritems():
        similar_user_books = user_book_matrix.loc[similar_user_id]
        rated_similar_user_books = similar_user_books.dropna()
        rated_similar_user_books = rated_similar_user_books[~rated_similar_user_books.index.isin(recommendations)]
        recommendations.extend(rated_similar_user_books.index)
        
        if len(recommendations) >= top_n:
            break
            
    return recommendations[:top_n]

# Пример использования функции для получения рекомендаций для пользователя с ID 276726
user_id = 276726
user_based_recommendations = get_top_n_recommendations_user_based(user_id)
print("User-Based Recommendations for User", user_id, ":", user_based_recommendations)

User-Based Recommendations for User 276726 : [' 9022906116', '0000000000', '00000000000', '0000001042283', '0000018030', '000104799X', '0001048759', '000105337X', '0001053736', '0001053744']


In [114]:
book_user_matrix = user_book_matrix.transpose()
book_similarity = cosine_similarity(book_user_matrix)
book_similarity_df = pd.DataFrame(book_similarity, index=book_user_matrix.index, columns=book_user_matrix.index)

# Функция для получения топ-N рекомендаций для пользователя на основе Item-based CF
def get_top_n_recommendations_item_based(user_id, top_n=10):
    user_books = user_book_matrix.loc[user_id]
    rated_books = user_books[user_books > 0]
    
    recommendations = []
    for book_id, rating in rated_books.iteritems():
        similar_books = book_similarity_df[book_id].sort_values(ascending=False)[1:]
        recommendations.extend(similar_books.index)
        
        if len(recommendations) >= top_n:
            break
            
    return recommendations[:top_n]

# Пример использования функции для получения рекомендаций для пользователя с ID 276726
item_based_recommendations = get_top_n_recommendations_item_based(user_id)
print("Item-Based Recommendations for User", user_id, ":", item_based_recommendations)


Item-Based Recommendations for User 276726 : [' 9022906116', '0743448162', '0743448642', '0743448677', '0743448782', '0743448944', '0743449002', '0743449150', '0743450884', '0743451414']
